In [1]:
import sys
import tensorflow as tf
import pandas as pd
import sklearn as sk
import numpy as np
import matplotlib

print(f"Python{sys.version}")
print()
print(f"Tensorflow version: {tf.__version__}")
print(f"Keras version: {tf.keras.__version__}")
print()
print(f"Pandas version: {pd.__version__}")
print(f"Scikit-Learn : {sk.__version__}")
print(f"Numpy version: {np.__version__}")
print(f"Matplotlib: {matplotlib.__version__}")
print()
print("GPU is ", "available" if tf.config.list_physical_devices('GPU') else "not available")
print(f"CUDA: {tf.test.is_built_with_cuda()}")

Python3.9.13 (main, Aug 25 2022, 23:51:50) [MSC v.1916 64 bit (AMD64)]

Tensorflow version: 2.11.0
Keras version: 2.11.0

Pandas version: 1.5.3
Scikit-Learn : 1.2.1
Numpy version: 1.24.2
Matplotlib: 3.7.0

GPU is  not available
CUDA: False


In [2]:
pwd


'E:\\10th Semester\\Computer Vision & Pattern Recognition\\Github\\Mid'